<h1><center>Coursera Capstone</center></h1>

Welcome to my Capstone Project. The purpose of this is to map generate a visual map of Toronto in Canada. Then we will proceed to a neighborhood comparison.

## Part II: Data Processing

This section is about data transformation and cleaning. 
1. Neighborhoods belonging to the same Borough are aggregated
1. Neighborhoods referenced by multiple Postal Codes must lie into each reference
1. Unlabeled Neighborhoods must take their respective Borough label

The existing dataset resulting from Part I will be extended by geocoding information

### Borough-level aggregation

In [27]:
import pandas as pd

df = pd.read_csv("./toronto_pc_conso.csv")

In this step:
1. We identify each unique borough
1. We compute Neighborhoods belonging to the same Borough into transitional DataFrames

Each unique Borough ends up with a list of Neighborhoods

In [28]:
col = [] 
upc = df.PostalCode.unique()
for b in upc:
    c = df[df['PostalCode'] == b]
    buffer = []
    strings = c['Neighborhood']
    for s in strings:
        q = df[df['Neighborhood'] == s]
        buffer.append(s)
    col.append(buffer)
    buffer = []
col

[['Parkwoods'],
 ['Victoria Village'],
 ['Harbourfront', 'Regent Park'],
 ['Lawrence Heights', 'Lawrence Manor'],
 ['Not assigned'],
 ['Islington Avenue'],
 ['Rouge', 'Malvern'],
 ['Don Mills North'],
 ['Woodbine Gardens', 'Parkview Hill'],
 ['Ryerson', 'Garden District'],
 ['Glencairn'],
 ['Cloverdale',
  'Islington',
  'Martin Grove',
  'Princess Gardens',
  'West Deane Park'],
 ['Highland Creek', 'Rouge Hill', 'Port Union'],
 ['Flemingdon Park', 'Don Mills South'],
 ['Woodbine Heights'],
 ['St. James Town'],
 ['Humewood-Cedarvale'],
 ['Bloordale Gardens', 'Eringate', 'Markland Wood', 'Old Burnhamthorpe'],
 ['Guildwood', 'Morningside', 'West Hill'],
 ['The Beaches'],
 ['Berczy Park'],
 ['Caledonia-Fairbanks'],
 ['Woburn'],
 ['Leaside'],
 ['Central Bay Street'],
 ['Christie'],
 ['Cedarbrae'],
 ['Hillcrest Village'],
 ['Bathurst Manor', 'Downsview North', 'Wilson Heights'],
 ['Thorncliffe Park'],
 ['Adelaide', 'King', 'Richmond'],
 ['Dovercourt Village', 'Dufferin'],
 ['Scarborough Vil

Each list is turned into a concatenated string

In [29]:
agg_nbh = []
for o in col:
    s = ','.join(o) 
    agg_nbh.append(s)

agg_nbh
len(agg_nbh)

103

Data exploration reveals that only two Neighborhoods have multiple Postal Codes. Furthermore, each Neighborhood has exactly two Postal Codes...

In [30]:
for a in df['Neighborhood']:
    #v = a.split(',')[0]
    b = df[df['Neighborhood'] == a]
    if(len(b)>1):
        print(len(b))
        print(b)

2
     Unnamed: 0 PostalCode           Borough    Neighborhood
27           27        M5C  Downtown Toronto  St. James Town
190         190        M4X  Downtown Toronto  St. James Town
2
     Unnamed: 0 PostalCode       Borough Neighborhood
115         115        M6N          York    Runnymede
145         145        M6S  West Toronto    Runnymede
2
     Unnamed: 0 PostalCode       Borough Neighborhood
115         115        M6N          York    Runnymede
145         145        M6S  West Toronto    Runnymede
2
     Unnamed: 0 PostalCode           Borough    Neighborhood
27           27        M5C  Downtown Toronto  St. James Town
190         190        M4X  Downtown Toronto  St. James Town


In [31]:
idx = []
for a in agg_nbh:
    v = a.split(',')[0]
    b = df[df['Neighborhood'] == v]
    if(len(b)>1):
        arr = b.index.values
        idx.append(arr[0])
        idx.append(arr[1])
idx

[27, 190, 115, 145]

Here we compute the Borough value of the first occurence in the aggregated Neighborhoods column. Then we insert the output into the list agg_bor.

In [32]:
agg_bor = []
for a in agg_nbh:
    v = a.split(',')[0]
    b = df[df['Neighborhood'] == v]
    w = b.index.values
    x = df.loc[w,'Borough'].values
    y = x[0]
    agg_bor.append(y)

Each list produced is casted into a new pandas Series.
We produce a new DataFrame by concatenating the Series.

In [33]:
x = pd.Series(upc)
y = pd.Series(agg_bor)
z = pd.Series(agg_nbh)
dfx = pd.concat([x,y,z],axis=1)
dfx.columns = ['PostalCode','Borough','Neighborhood']
dfx

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [34]:
dfx['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
York                 6
East York            5
West Toronto         5
East Toronto         5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

Let's get the index of unassigned Neighborhoods

In [35]:
dfx[dfx['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
4,M7A,Queen's Park,Not assigned


In [36]:
dfx.loc[4,'Neighborhood'] = dfx.loc[4,'Borough'] 
dfx

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### Dataset extension

While following the assignment guidelines has been quite straightforward, extending the dataset with geocoding turned out to be painful. This is due to the unreliability of the geocoder library that we have been assigned to use. Googles' proprietary geocoder is paid, and pyhton geopy ends up with a request timeout response.
To cope with this issue, we rely on the Foursquare API.

### The Foursquare API

for the rest of this section, I will assume that you already have your own Foursquare credentials. If not, please make sure you create a Foursquare account and store your own credentials in file named 'creds.json'. The JSON file must be on the same working directory as the notebook and respect the following structure:

In [37]:
{
    "client_id": "your-id"
    "client_secret": "your-secret"
    "version": "20180604"
}

SyntaxError: invalid syntax (<ipython-input-37-c991d475691e>, line 3)

In [38]:
import json

with open("creds.json","r") as file:
    f = json.load(file)
    client_id = f.get("client_id")
    client_secret = f.get("client_secret")
    version = f.get("version")


#### Querying the API

In [39]:
resps = []
for pc in upc:
    near = str(pc) + ', Toronto, Ontario'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&near={}'.format(client_id, client_secret, version,near)
    response = requests.get(url).json()
    resps.append(response)

In [ ]:
resps

In [41]:
queries = []
for q in resps:
    queries.append(q['response'])

As expected, there is one query dictionary by referenced Borough

In [42]:
len(queries)

103

To extract the geocoding information, we need to dive into the geocode block of the response.

In [43]:
venues = []
geocode = []
for item in queries:
    v = item.get("venues")
    g = item.get("geocode")
    venues.append(v)
    geocode.append(g)

In [63]:
print('(venues: {}, geocodes: {})'.format((len(venues)),len(geocode)))

(venues: 103, geocodes: 102)


Venues output corresponds to the expected format, but it seems that there is a missing value in the geocode list.
1. We export the venues for further use
1. We run the geocode loop to extract lat,lng values from each geocode block 

In [44]:
with open("venues.json", "w") as wf:
    for venue in venues:
        json.dump(venue, wf)

In [45]:
lats = []
lngs = []
names = []
for item in geocode:
    if (item is None):
        geocode.pop(geocode.index(item))
    for uid,code in item.items():
        print(uid)
        print(code)
        if(uid == 'feature'):
            lats.append(code.get("geometry").get("center").get('lat'))
            lngs.append(code.get("geometry").get("center").get("lng"))
            names.append(code.get("name"))
        

what

where
m3a toronto ontario
feature
{'cc': 'CA', 'name': 'M3A', 'displayName': 'M3A, Canada', 'matchedName': 'M3A, Toronto, Ontario, Canada', 'highlightedName': '<b>M3A</b>, <b>Toronto</b>, <b>Ontario</b>, Canada', 'woeType': 11, 'id': 'geonamezip:CA-M3A', 'longId': '146648462866283668', 'geometry': {'center': {'lat': 43.7545, 'lng': -79.33}}}
parents
[]
what

where
m4a toronto ontario
feature
{'cc': 'CA', 'name': 'M4A', 'displayName': 'M4A, Canada', 'matchedName': 'M4A, Toronto, Ontario, Canada', 'highlightedName': '<b>M4A</b>, <b>Toronto</b>, <b>Ontario</b>, Canada', 'woeType': 11, 'id': 'geonamezip:CA-M4A', 'longId': '146648462866283706', 'geometry': {'center': {'lat': 43.7276, 'lng': -79.3148}}}
parents
[]
what

where
m5a toronto ontario
feature
{'cc': 'CA', 'name': 'M5A', 'displayName': 'M5A, Canada', 'matchedName': 'M5A, Toronto, Ontario, Canada', 'highlightedName': '<b>M5A</b>, <b>Toronto</b>, <b>Ontario</b>, Canada', 'woeType': 11, 'id': 'geonamezip:CA-M5A', 'longId': '1466

AttributeError: 'NoneType' object has no attribute 'items'

In [46]:
print('({},{})'.format((len(lats)),len(lngs)))

(76,76)


Indeed, the 77th entry is missing in the geocode list. Thus, we need to skip the 77th computation in order to capture the whole information. This can be achieved through a drop of the entry at the 77th position in the geocode list.

In [65]:
geocode[75:77]

[{'what': '',
  'where': 'm6r toronto ontario',
  'feature': {'cc': 'CA',
   'name': 'M6R',
   'displayName': 'M6R, Canada',
   'matchedName': 'M6R, Toronto, Ontario, Canada',
   'highlightedName': '<b>M6R</b>, <b>Toronto</b>, <b>Ontario</b>, Canada',
   'woeType': 11,
   'id': 'geonamezip:CA-M6R',
   'longId': '146648462866283799',
   'geometry': {'center': {'lat': 43.6469, 'lng': -79.4521}}},
  'parents': []},
 {'what': '',
  'where': 'm9r toronto ontario',
  'feature': {'cc': 'CA',
   'name': 'M9R',
   'displayName': 'M9R, Canada',
   'matchedName': 'M9R, Toronto, Ontario, Canada',
   'highlightedName': '<b>M9R</b>, <b>Toronto</b>, <b>Ontario</b>, Canada',
   'woeType': 11,
   'id': 'geonamezip:CA-M9R',
   'longId': '146648462866283913',
   'geometry': {'center': {'lat': 43.6898, 'lng': -79.5582}}},
  'parents': []}]

In [67]:
dfx.loc[75:77,['PostalCode']]

,PostalCode
75,M6R
76,M7R
77,M9R


In [74]:
# the pop command must not be tried more than once, so we store geocode for safety
copy = geocode.copy() 
geocode.pop(76)

{'what': '',
 'where': 'm6r toronto ontario',
 'feature': {'cc': 'CA',
  'name': 'M6R',
  'displayName': 'M6R, Canada',
  'matchedName': 'M6R, Toronto, Ontario, Canada',
  'highlightedName': '<b>M6R</b>, <b>Toronto</b>, <b>Ontario</b>, Canada',
  'woeType': 11,
  'id': 'geonamezip:CA-M6R',
  'longId': '146648462866283799',
  'geometry': {'center': {'lat': 43.6469, 'lng': -79.4521}}},
 'parents': []}

Then we simply run again the geocode loop.
1. Select the feature object
1. Select the geometry object
1. Select the center object

...and get the Lat,Long data.

In [75]:
lats = []
lngs = []
names = []
for item in geocode:
    if (item is None):
        geocode.pop(geocode.index(item))
    for uid,code in item.items():
        print(uid)
        print(code)
        if(uid == 'feature'):
            lats.append(code.get("geometry").get("center").get('lat'))
            lngs.append(code.get("geometry").get("center").get("lng"))
            names.append(code.get("name"))

what

where
m3a toronto ontario
feature
{'cc': 'CA', 'name': 'M3A', 'displayName': 'M3A, Canada', 'matchedName': 'M3A, Toronto, Ontario, Canada', 'highlightedName': '<b>M3A</b>, <b>Toronto</b>, <b>Ontario</b>, Canada', 'woeType': 11, 'id': 'geonamezip:CA-M3A', 'longId': '146648462866283668', 'geometry': {'center': {'lat': 43.7545, 'lng': -79.33}}}
parents
[]
what

where
m4a toronto ontario
feature
{'cc': 'CA', 'name': 'M4A', 'displayName': 'M4A, Canada', 'matchedName': 'M4A, Toronto, Ontario, Canada', 'highlightedName': '<b>M4A</b>, <b>Toronto</b>, <b>Ontario</b>, Canada', 'woeType': 11, 'id': 'geonamezip:CA-M4A', 'longId': '146648462866283706', 'geometry': {'center': {'lat': 43.7276, 'lng': -79.3148}}}
parents
[]
what

where
m5a toronto ontario
feature
{'cc': 'CA', 'name': 'M5A', 'displayName': 'M5A, Canada', 'matchedName': 'M5A, Toronto, Ontario, Canada', 'highlightedName': '<b>M5A</b>, <b>Toronto</b>, <b>Ontario</b>, Canada', 'woeType': 11, 'id': 'geonamezip:CA-M5A', 'longId': '1466

In [76]:
print('({},{})'.format((len(lats)),len(lngs)))

(102,102)


This step consists on inserting NaN values into the geocoding lists to align it with the dfx DataFrame's index.
Then both lists are converted into pandas Series and concatenated into a new DataFrame dfy

In [77]:
import numpy as np
lats.insert(76,np.nan)
lngs.insert(76,np.nan)

In [78]:
lt = pd.Series(lats)
lg = pd.Series(lngs)
dfy = pd.concat([lt,lg],axis = 1)
dfy.columns = ['Latitude','Longitude']
dfy

,Latitude,Longitude
0,43.7545,-79.3300
1,43.7276,-79.3148
2,43.6555,-79.3626
3,43.7223,-79.4504
4,43.6641,-79.3889
5,43.6662,-79.5282
6,43.8113,-79.1930
7,43.7450,-79.3590
8,43.7063,-79.3094
9,43.6572,-79.3783


In [80]:
dataset = pd.concat([dfx,dfy],axis = 1)
dataset

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6555,-79.3626
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.7223,-79.4504
4,M7A,Queen's Park,Queen's Park,43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Rouge,Malvern",43.8113,-79.1930
7,M3B,North York,Don Mills North,43.7450,-79.3590
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.7063,-79.3094
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3783


In [81]:
dataset.to_csv('./toronto_pc_agg.csv')

DataFrames dfx and dfy are concatenated into the DataFrame dataset. Which we export as csv for reusability.